In [4]:
import googlemaps
import pprint
import time
from googleapikey import get_my_api_key
import json
import xlsxwriter
from bs4 import BeautifulSoup
import pandas as pd
import glob

In [255]:
# Define API Key
API_KEY = get_my_api_key

# Define CLient
gmaps = googlemaps.Client(key=API_KEY)

# Define Search
places_result = gmaps.places_nearby(location="41.4993, -81.6944", radius=40000, open_now=False, name="TacoBell", 
type="restaurant")

time.sleep(3)

# place_result  = gmaps.places_nearby(page_token = places_result['next_page_token'])

weekday = []
address = []
name = []
url = []

# Loop through results Place IDs
for place in places_result["results"]:
    # Define each Place ID
    my_place_id = place["place_id"]
    # Define fields sent back
    my_fields = ["name", "adr_address", "opening_hours", "url"]
    # Make a request for the details
    places_details = gmaps.place(place_id=my_place_id, fields=my_fields)
    # Print the results
    pprint.pprint(places_details["result"])
    print("------")
    try:
        address.append([places_details["result"]["adr_address"]])
    except:
        continue
    try:
        name.append(places_details["result"]["name"])
    except:
        continue
    try:
        weekday.append(places_details["result"]["opening_hours"]["weekday_text"])
    except:
        continue
    try:
        url.append(places_details["result"]["url"])
    except:
        continue


{'adr_address': '<span class="street-address">3776 W 150th St</span>, <span '
                'class="locality">Cleveland</span>, <span '
                'class="region">OH</span> <span '
                'class="postal-code">44111</span>, <span '
                'class="country-name">USA</span>',
 'name': 'Taco Bell',
 'opening_hours': {'open_now': True,
                   'periods': [{'close': {'day': 1, 'time': '0500'},
                                'open': {'day': 0, 'time': '0700'}},
                               {'close': {'day': 2, 'time': '0500'},
                                'open': {'day': 1, 'time': '0700'}},
                               {'close': {'day': 3, 'time': '0500'},
                                'open': {'day': 2, 'time': '0700'}},
                               {'close': {'day': 4, 'time': '0500'},
                                'open': {'day': 3, 'time': '0700'}},
                               {'close': {'day': 5, 'time': '0500'},
                      

{'adr_address': '<span class="street-address">117 West Ave</span>, <span '
                'class="locality">Elyria</span>, <span '
                'class="region">OH</span> <span '
                'class="postal-code">44035</span>, <span '
                'class="country-name">USA</span>',
 'name': 'Taco Bell',
 'opening_hours': {'open_now': True,
                   'periods': [{'close': {'day': 2, 'time': '0000'},
                                'open': {'day': 1, 'time': '0700'}},
                               {'close': {'day': 3, 'time': '0000'},
                                'open': {'day': 2, 'time': '0700'}},
                               {'close': {'day': 4, 'time': '0000'},
                                'open': {'day': 3, 'time': '0700'}},
                               {'close': {'day': 5, 'time': '0000'},
                                'open': {'day': 4, 'time': '0700'}},
                               {'close': {'day': 6, 'time': '0000'},
                            

{'adr_address': '<span class="street-address">1531 W 117th St</span>, <span '
                'class="locality">Cleveland</span>, <span '
                'class="region">OH</span> <span '
                'class="postal-code">44107</span>, <span '
                'class="country-name">USA</span>',
 'name': 'Taco Bell',
 'opening_hours': {'open_now': True,
                   'periods': [{'close': {'day': 1, 'time': '0300'},
                                'open': {'day': 0, 'time': '0800'}},
                               {'close': {'day': 2, 'time': '0300'},
                                'open': {'day': 1, 'time': '0800'}},
                               {'close': {'day': 3, 'time': '0300'},
                                'open': {'day': 2, 'time': '0800'}},
                               {'close': {'day': 4, 'time': '0300'},
                                'open': {'day': 3, 'time': '0800'}},
                               {'close': {'day': 5, 'time': '0300'},
                      

{'adr_address': '<span class="street-address">24247 Lorain Rd</span>, <span '
                'class="locality">North Olmsted</span>, <span '
                'class="region">OH</span> <span '
                'class="postal-code">44070</span>, <span '
                'class="country-name">USA</span>',
 'name': 'Taco Bell',
 'opening_hours': {'open_now': True,
                   'periods': [{'close': {'day': 1, 'time': '0300'},
                                'open': {'day': 0, 'time': '0800'}},
                               {'close': {'day': 2, 'time': '0300'},
                                'open': {'day': 1, 'time': '0800'}},
                               {'close': {'day': 3, 'time': '0300'},
                                'open': {'day': 2, 'time': '0800'}},
                               {'close': {'day': 4, 'time': '0300'},
                                'open': {'day': 3, 'time': '0800'}},
                               {'close': {'day': 5, 'time': '0300'},
                  

In [256]:
new_address = []
for i in address:
    
    
    for j in i:
        soup = BeautifulSoup(j, 'html.parser')    
    #warmup getting tickers on first page before looping over other pages
        try:
            st = [i.text for i in soup.find_all('span', class_='street-address')][0]
        except:
            continue
        try:
            loc = [i.text for i in soup.find_all('span', class_='locality')][0]
        except:
            continue
        try:
            reg = [i.text for i in soup.find_all('span', class_='region')][0]
        except:
            continue
        try:
            post = [i.text for i in soup.find_all('span', class_='postal-code')][0]
        except:
            continue
    temp = [st, loc, reg, post]
#     temp = {"street-address": st, "locality": loc, "region": reg, "postal-code": post}
    new_address.append(temp)
df = pd.DataFrame(data = [new_address,weekday,name,url])
df = df.transpose().copy()
df.rename({0:"Address", 1:"Time", 2:"Name", 3:"Url"},axis=1, inplace=True)
for i, j in enumerate(df["Time"]):
    for day in j:
        df.loc[i, str(day[:3])] = day.split("y")[1][2:]
        
df = df[["Name", "Address", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun", "Url"]]

In [257]:
df

,Name,Address,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Url
0,Taco Bell,"[3776 W 150th St, Cleveland, OH, 44111]",7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,https://maps.google.com/?cid=2565244456744257602
1,Taco Bell,"[4110 Lee Rd, Cleveland, OH, 44128]",9:00 AM – 2:00 AM,9:00 AM – 2:00 AM,9:00 AM – 2:00 AM,9:00 AM – 3:00 AM,9:00 AM – 3:00 AM,9:00 AM – 3:00 AM,9:00 AM – 2:00 AM,https://maps.google.com/?cid=12662104814896757838
2,Taco Bell,"[17565 Bagley Rd, Middleburg Heights, OH, 44130]",7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 5:00 AM,7:00 AM – 4:00 AM,https://maps.google.com/?cid=4284472979876242945
3,Taco Bell,"[26002 Euclid Ave, Euclid, OH, 44132]",8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,https://maps.google.com/?cid=271874603143044544
4,Taco Bell,"[14701 Snow Rd, Brook Park, OH, 44142]",9:00 AM – 2:00 AM,9:00 AM – 2:00 AM,9:00 AM – 2:00 AM,9:00 AM – 2:00 AM,9:00 AM – 3:00 AM,9:00 AM – 3:00 AM,9:00 AM – 2:00 AM,https://maps.google.com/?cid=5008302405188147974
5,Taco Bell,"[5591 Warrensville Center Rd, Maple Heights, O...",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 3:00 AM,https://maps.google.com/?cid=9185639906944721922
6,Taco Bell,"[10800 Lorain Ave, Cleveland, OH, 44111]",8:00 AM – 2:00 AM,8:00 AM – 2:00 AM,8:00 AM – 2:00 AM,8:00 AM – 2:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,9:15 AM – 2:00 AM,https://maps.google.com/?cid=9514553419716727394
7,Taco Bell,"[117 West Ave, Elyria, OH, 44035]",7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,Closed,https://maps.google.com/?cid=13787063022319263108
8,Taco Bell,"[5780 Broadview Rd, Parma, OH, 44134]",8:00 AM – 2:00 AM,8:00 AM – 2:00 AM,8:00 AM – 2:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 2:00 AM,https://maps.google.com/?cid=14680000787958375406
9,Taco Bell,"[9486 Mentor Ave, Mentor, OH, 44060]",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,https://maps.google.com/?cid=7776322742558664657


In [47]:
df = df[df.Name == "Pizza Hut"]
df

,Address,Time,Name,Url,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,"[24750 Telegraph Rd, Southfield, MI, 48033]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=13169877648311171048,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 11:00 PM,11:00 AM – 10:00 PM
1,"[31315 Van Dyke Ave, Warren, MI, 48093]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=1838141604267172129,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 12:00 AM,11:00 AM – 10:00 PM
2,"[23137 Woodward Ave, Ferndale, MI, 48220]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=15594166808002514118,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 11:00 PM,11:00 AM – 10:00 PM
3,"[2941 S Telegraph Rd, Dearborn, MI, 48124]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=7502631056950440423,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 11:00 PM,12:00 – 10:00 PM
4,"[25240 Gratiot Ave, Roseville, MI, 48066]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=10523788749371012020,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 12:00 AM,11:00 AM – 10:00 PM
5,"[43443 Grand River Ave Ste 220, Novi, MI, 48375]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=10888074127936186746,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 12:00 AM,11:00 AM – 10:00 PM
6,"[56 14 Mile Rd, Madison Heights, MI, 48071]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=10768261185347457125,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 12:00 AM,11:00 AM – 10:00 PM
7,"[10165 Telegraph Rd, Taylor, MI, 48180]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=4406768023243967961,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 12:00 AM,11:00 AM – 12:00 AM,11:00 AM – 10:00 PM


In [100]:
df.dropna(inplace=True)
df

,Address,Time,Name,Url,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,"[4989 Lake Michigan Dr, Allendale Charter Twp,...","[Monday: 7:00 AM – 12:00 AM, Tuesday: 7:00 AM ...",Taco Bell,https://maps.google.com/?cid=18206311437575694958,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM
1,"[190 N River Ave, Holland, MI, 49424]","[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...",Taco Bell,https://maps.google.com/?cid=3630250545044647256,6:00 AM – 1:00 AM,6:00 AM – 1:00 AM,6:00 AM – 1:00 AM,6:00 AM – 1:00 AM,6:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM
2,"[188 Chicago Dr, Jenison, MI, 49428]","[Monday: 7:00 AM – 12:00 AM, Tuesday: 7:00 AM ...",Taco Bell,https://maps.google.com/?cid=3348183267406193748,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 10:00 PM,7:00 AM – 12:00 AM
3,"[2829 28th St SE, Grand Rapids, MI, 49512-1622]","[Monday: 7:00 AM – 10:30 PM, Tuesday: 7:00 AM ...",Taco Bell,https://maps.google.com/?cid=10715835748954320271,7:00 AM – 10:30 PM,7:00 AM – 10:30 PM,7:00 AM – 10:30 PM,7:00 AM – 10:30 PM,7:00 AM – 10:30 PM,7:00 AM – 10:30 PM,7:00 AM – 10:30 PM
4,"[4234 Lake Michigan Dr NW, Grand Rapids, MI, 4...","[Monday: 7:00 AM – 10:00 PM, Tuesday: 7:00 AM ...",Taco Bell,https://maps.google.com/?cid=10554313471804595390,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM
5,"[3160 Alpine Ave NW, Grand Rapids, MI, 49544]","[Monday: 7:00 AM – 1:00 AM, Tuesday: 7:00 AM –...",Taco Bell,https://maps.google.com/?cid=9309234869226258901,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 2:00 AM,7:00 AM – 2:00 AM,7:00 AM – 1:00 AM
6,"[1014 Michigan St NE, Grand Rapids, MI, 49503]","[Monday: 9:00 AM – 9:00 PM, Tuesday: 9:00 AM –...",Taco Bell,https://maps.google.com/?cid=9898107988034337401,9:00 AM – 9:00 PM,9:00 AM – 9:00 PM,9:00 AM – 9:00 PM,9:00 AM – 9:00 PM,9:00 AM – 9:00 PM,9:00 AM – 9:00 PM,9:00 AM – 9:00 PM
7,"[695 68th St SW, Byron Center, MI, 49315]","[Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM –...",Taco Bell,https://maps.google.com/?cid=7646751925993475684,7:00 AM – 8:00 PM,7:00 AM – 8:00 PM,7:00 AM – 8:00 PM,7:00 AM – 8:00 PM,7:00 AM – 8:00 PM,7:00 AM – 8:00 PM,7:00 AM – 8:00 PM
8,"[5049 Northland Dr NE, Grand Rapids, MI, 49525]","[Monday: 7:00 AM – 12:00 AM, Tuesday: 7:00 AM ...",Taco Bell,https://maps.google.com/?cid=11095978351589494530,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM
9,"[6010 28th St SE, Grand Rapids, MI, 49546]","[Monday: 7:00 AM – 10:00 PM, Tuesday: 7:00 AM ...",Taco Bell,https://maps.google.com/?cid=16643185705083107748,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM,7:00 AM – 10:00 PM


In [252]:
df = df.drop([7, 14, 16, 17, 18, 19])
df

,Address,Time,Name,Url,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,"[10616 Lorain Ave, Cleveland, OH, 44111]","[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=943797734015333453,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 11:30 PM,11:00 AM – 11:30 PM,11:00 AM – 10:30 PM
1,"[6428 Pearl Rd, Parma Heights, OH, 44130]","[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=2918654493793979871,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 11:30 PM,11:00 AM – 11:30 PM,11:00 AM – 10:30 PM
2,"[18055 Bagley Rd, Middleburg Heights, OH, 44130]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=18239478218849822619,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 10:00 PM
3,"[16800 Chagrin Blvd, Shaker Heights, OH, 44120]","[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=7502478856794058266,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 10:30 PM,11:00 AM – 11:30 PM,11:00 AM – 11:30 PM,11:00 AM – 10:30 PM
4,"[6557 Broadway Ave, Cleveland, OH, 44105]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=9549280812009604946,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 8:30 PM,11:00 AM – 10:00 PM
5,"[27845 Lorain Rd, North Olmsted, OH, 44070]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=13846002040765221031,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 10:00 PM
6,"[5041 Abbe Rd N, Elyria, OH, 44035]","[Monday: 11:00 AM – 11:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=4139995929871894051,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 12:00 AM,11:00 AM – 12:00 AM,11:00 AM – 11:00 PM
8,"[7002 Biddulph Rd, Brooklyn, OH, 44144]","[Monday: Closed, Tuesday: 11:00 AM – 9:00 PM, ...",Pizza Hut,https://maps.google.com/?cid=13443162757928183988,Closed,11:00 AM – 9:00 PM,11:00 AM – 8:00 PM,11:00 AM – 8:00 PM,11:00 AM – 8:00 PM,11:00 AM – 9:00 PM,4:00 – 9:00 PM
9,"[544 Water St, Chardon, OH, 44024]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=15532200400843181846,11:00 AM – 10:00 PM,11:00 AM – 10:45 PM,11:00 AM – 10:45 PM,11:00 AM – 10:45 PM,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 10:00 PM
10,"[6550 Royalton Rd, North Royalton, OH, 44133]","[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",Pizza Hut,https://maps.google.com/?cid=10854583871893478426,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 10:00 PM,11:00 AM – 11:00 PM,11:00 AM – 11:00 PM,11:00 AM – 10:00 PM


In [181]:
df

,Address,Time,Name,Url,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,"[1341 Victories Ln, Petoskey, MI, 49770]","[Monday: 5:30 AM – 8:00 PM, Tuesday: 5:30 AM –...",Starbucks,https://maps.google.com/?cid=15012412240483820312,5:30 AM – 8:00 PM,5:30 AM – 8:00 PM,5:30 AM – 8:00 PM,5:30 AM – 8:00 PM,5:30 AM – 8:00 PM,5:30 AM – 8:00 PM,5:30 AM – 8:00 PM


In [258]:
df = df[["Name", "Address", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun", "Url"]]

In [259]:
df.to_excel("apihourdata/cleve_taco.xlsx", index=False)

In [ ]:
############################

In [265]:
# Merge
path = "C:/Users/Westjohn/Desktop/FC/KPI Trackers/apihourdata"
file_list = glob.glob(path + "/*.xlsx")

excl_list = []

for file in file_list:
    excl_list.append(pd.read_excel(file))
    
excl_merged = pd.DataFrame()

for excl_file in excl_list:
    excl_merged = excl_merged.append(excl_file, ignore_index=True)
    
excl_merged.to_excel("apihourdata/grouped.xlsx", index=False)


In [266]:
df = pd.read_excel("apihourdata/grouped.xlsx")
df

,Nearby_City,Name,Address,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Url
0,"Charlevoix, MI",KFC,"['1191 U.S. 31 N', 'Petoskey', 'MI', '49770']",7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 3:00 AM,7:00 AM – 3:00 AM,7:00 AM – 1:00 AM,https://maps.google.com/?cid=5255748225952308909
1,"Charlevoix, MI",McDonald's,"['1447 W South Airport Rd', 'Traverse City', '...",6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,https://maps.google.com/?cid=18446472885863542548
2,"Charlevoix, MI",McDonald's,"['747 Spring St US 131', 'Petoskey', 'MI', '49...",6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,https://maps.google.com/?cid=9892539451885645870
3,"Charlevoix, MI",McDonald's,"['1440 Bridge St', 'Charlevoix', 'MI', '49720']",6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,6:00 AM – 11:00 PM,https://maps.google.com/?cid=3825878298203522926
4,"Charlevoix, MI",McDonald's,"['1112 Boyne Ave', 'Boyne City', 'MI', '49712']",6:00 AM – 10:00 PM,6:00 AM – 10:00 PM,6:00 AM – 10:00 PM,6:00 AM – 10:00 PM,6:00 AM – 10:00 PM,6:00 AM – 10:00 PM,6:00 AM – 10:00 PM,https://maps.google.com/?cid=7883555982371308564
...,...,...,...,...,...,...,...,...,...,...,...
471,"Traverse City, MI",Starbucks,"['1120 E Front St', 'Traverse City', 'MI', '49...",5:30 AM – 9:00 PM,5:30 AM – 9:00 PM,5:30 AM – 9:00 PM,5:30 AM – 9:00 PM,5:30 AM – 9:00 PM,6:00 AM – 9:00 PM,6:00 AM – 9:00 PM,https://maps.google.com/?cid=14926678828292246408
472,"Traverse City, MI",Taco Bell,"['816 E Front St', 'Traverse City', 'MI', '496...",7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 12:00 AM,https://maps.google.com/?cid=14581314721087642623
473,"Traverse City, MI",Taco Bell,"['1447 W South Airport Rd', 'Traverse City', '...",7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 12:00 AM,https://maps.google.com/?cid=12312107052389907204
474,"Traverse City, MI",Taco Bell,"['1447 W South Airport Rd', 'Traverse City', '...",7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 12:00 AM,7:00 AM – 1:00 AM,7:00 AM – 1:00 AM,7:00 AM – 12:00 AM,https://maps.google.com/?cid=1862214899328130276


In [262]:
df = df.assign(Nearby_City="Cleveland, OH")
df

,Name,Address,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Url,Nearby_City
0,KFC,"['38315 Chestnut Ridge Rd', 'Elyria', 'OH', '4...",10:30 AM – 8:00 PM,10:30 AM – 8:00 PM,10:30 AM – 8:00 PM,10:30 AM – 8:00 PM,10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 8:00 PM,https://maps.google.com/?cid=13138280041200353380,"Cleveland, OH"
1,KFC,"['5380 Pearl Rd', 'Parma', 'OH', '44129']",10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 11:00 PM,10:30 AM – 11:00 PM,10:30 AM – 10:00 PM,https://maps.google.com/?cid=10245356699819790493,"Cleveland, OH"
2,KFC,"['1560 W 117th St', 'Lakewood', 'OH', '44107']",10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,https://maps.google.com/?cid=12677371725199570971,"Cleveland, OH"
3,KFC,"['2930 Carnegie Ave', 'Cleveland', 'OH', '44115']",10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,https://maps.google.com/?cid=16515467442592915391,"Cleveland, OH"
4,KFC,"['307 Highland Rd', 'Macedonia', 'OH', '44056']",10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 9:00 PM,https://maps.google.com/?cid=16437554875584852990,"Cleveland, OH"
...,...,...,...,...,...,...,...,...,...,...,...
81,Taco Bell,"['15500 Detroit Ave', 'Lakewood', 'OH', '44107']",8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 5:00 AM,8:00 AM – 5:00 AM,8:00 AM – 4:00 AM,https://maps.google.com/?cid=6826661475303145302,"Cleveland, OH"
82,Taco Bell,"['993 Graham Rd', 'Cuyahoga Falls', 'OH', '442...",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,https://maps.google.com/?cid=5247291276782185481,"Cleveland, OH"
83,Taco Bell,"['557 E Aurora Rd', 'Macedonia', 'OH', '44056']",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 3:00 AM,https://maps.google.com/?cid=4326910596614455051,"Cleveland, OH"
84,Taco Bell,"['24247 Lorain Rd', 'North Olmsted', 'OH', '44...",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 3:00 AM,https://maps.google.com/?cid=3676959712853323956,"Cleveland, OH"


In [263]:
df = df[["Nearby_City", "Name", "Address", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun", "Url"]]
df

,Nearby_City,Name,Address,Mon,Tue,Wed,Thu,Fri,Sat,Sun,Url
0,"Cleveland, OH",KFC,"['38315 Chestnut Ridge Rd', 'Elyria', 'OH', '4...",10:30 AM – 8:00 PM,10:30 AM – 8:00 PM,10:30 AM – 8:00 PM,10:30 AM – 8:00 PM,10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 8:00 PM,https://maps.google.com/?cid=13138280041200353380
1,"Cleveland, OH",KFC,"['5380 Pearl Rd', 'Parma', 'OH', '44129']",10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 11:00 PM,10:30 AM – 11:00 PM,10:30 AM – 10:00 PM,https://maps.google.com/?cid=10245356699819790493
2,"Cleveland, OH",KFC,"['1560 W 117th St', 'Lakewood', 'OH', '44107']",10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,https://maps.google.com/?cid=12677371725199570971
3,"Cleveland, OH",KFC,"['2930 Carnegie Ave', 'Cleveland', 'OH', '44115']",10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,https://maps.google.com/?cid=16515467442592915391
4,"Cleveland, OH",KFC,"['307 Highland Rd', 'Macedonia', 'OH', '44056']",10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 9:00 PM,10:30 AM – 10:00 PM,10:30 AM – 10:00 PM,10:30 AM – 9:00 PM,https://maps.google.com/?cid=16437554875584852990
...,...,...,...,...,...,...,...,...,...,...,...
81,"Cleveland, OH",Taco Bell,"['15500 Detroit Ave', 'Lakewood', 'OH', '44107']",8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 5:00 AM,8:00 AM – 5:00 AM,8:00 AM – 4:00 AM,https://maps.google.com/?cid=6826661475303145302
82,"Cleveland, OH",Taco Bell,"['993 Graham Rd', 'Cuyahoga Falls', 'OH', '442...",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,https://maps.google.com/?cid=5247291276782185481
83,"Cleveland, OH",Taco Bell,"['557 E Aurora Rd', 'Macedonia', 'OH', '44056']",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 3:00 AM,https://maps.google.com/?cid=4326910596614455051
84,"Cleveland, OH",Taco Bell,"['24247 Lorain Rd', 'North Olmsted', 'OH', '44...",8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 3:00 AM,8:00 AM – 4:00 AM,8:00 AM – 4:00 AM,8:00 AM – 3:00 AM,https://maps.google.com/?cid=3676959712853323956


In [8]:
df.columns

Index(['Nearby_City', 'Name', 'Address', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri',
       'Sat', 'Sun', 'Url'],
      dtype='object')

In [267]:
df.to_excel("apihourdata/complete.xlsx", index=False)